In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 125
import seaborn as sns
from Bio import SeqIO, Seq
import scipy.stats as st
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import glob, os, yaml, subprocess, itertools, sparse, pickle

who_variants = pd.read_csv("/n/data1/hms/dbmi/farhat/Sanjana/MIC_data/WHO_resistance_variants_all.csv")
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Combined Analysis Files

In [30]:
drug = "Delamanid"
analysis = pd.read_csv(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/{drug}/final_analysis.csv")

# drug = "Levofloxacin"
# lev_analysis = pd.read_csv(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/{drug}/final_analysis.csv")

# drug = "Amikacin"
# ami_analysis = pd.read_csv(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/{drug}/final_analysis.csv")

In [31]:
analysis.columns

Index(['orig_variant', 'coef', 'coef_LB', 'coef_UB', 'pval', 'BH_pval',
       'Bonferroni_pval', 'genome_index', 'confidence_WHO_2021', 'Odds_Ratio',
       'OR_LB', 'OR_UB', 'Tier1_only', 'WHO_phenos', 'poolLOF', 'Syn',
       'Num_Isolates', 'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens',
       'Spec', 'LR+', 'LR-', 'PPV_LB', 'PPV_UB', 'Sens_LB', 'Sens_UB',
       'Spec_LB', 'Spec_UB', 'LR+_LB', 'LR+_UB', 'LR-_LB', 'LR-_UB'],
      dtype='object')

In [32]:
analysis

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,PPV_LB,PPV_UB,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB
0,ddn_p.Asp108fs,0.2241,0.0000,0.2830,0.0002,0.0151,0.0151,NaN,NaN,1.2512,...,1.0000,1.0000,0.0000,0.0269,1.0000,1.0000,inf,inf,0.9731,1.0000
1,ddn_p.Val127Phe,0.1563,-0.0000,0.2306,0.0025,0.0348,0.1742,NaN,NaN,1.1692,...,0.0000,1.0000,0.0000,0.0272,0.9996,1.0000,0.0000,inf,0.9730,1.0002
2,ddn_p.Leu49Pro,0.1377,0.1171,0.1539,0.0000,0.0000,0.0000,3986989.0000,2) Assoc w R - Interim,1.1476,...,0.6154,1.0000,0.0215,0.0723,0.9996,1.0000,77.3261,inf,0.9278,0.9786
3,fbiB_p.Lys448Arg,0.1352,-0.0469,0.2926,0.0575,0.1203,1.0000,3642877.0000,3) Uncertain significance,1.1448,...,0.0641,0.1633,0.0376,0.1051,0.9863,0.9902,3.2542,8.9502,0.9069,0.9734
4,ddn_p.Trp139*,0.1268,-0.0000,0.1689,0.0354,0.1061,1.0000,NaN,NaN,1.1352,...,1.0000,1.0000,0.0000,0.0272,1.0000,1.0000,inf,inf,0.9728,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,fbiC_c.-32A>G,-0.0645,-0.0986,-0.0207,0.0004,0.0153,0.0306,1302899.0000,3) Uncertain significance,0.9376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,fbiC_c.967C>T,-0.0646,-0.0753,-0.0518,0.0000,0.0000,0.0000,NaN,NaN,0.9374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,fbiB_p.Leu447Arg,-0.0649,-0.0790,-0.0497,0.0000,0.0000,0.0000,3642874.0000,3) Uncertain significance,0.9371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,fgd1_c.465C>T,-0.0783,-0.0907,-0.0649,0.0000,0.0000,0.0000,NaN,NaN,0.9247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
analysis.columns

Index(['orig_variant', 'coef', 'coef_LB', 'coef_UB', 'pval', 'BH_pval',
       'Bonferroni_pval', 'genome_index', 'confidence_WHO_2021', 'Odds_Ratio',
       'OR_LB', 'OR_UB', 'Tier1_only', 'WHO_phenos', 'poolLOF', 'Syn',
       'Num_Isolates', 'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens',
       'Spec', 'LR+', 'LR-', 'PPV_LB', 'PPV_UB', 'Sens_LB', 'Sens_UB',
       'Spec_LB', 'Spec_UB', 'LR+_LB', 'LR+_UB', 'LR-_LB', 'LR-_UB'],
      dtype='object')

In [27]:
analysis.query("confidence_WHO_2021=='2) Assoc w R - Interim'")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,PPV_LB,PPV_UB,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB
69,ddn_p.Leu49Pro,0.1377,0.1171,0.1539,0.0000,0.0000,0.0000,3986989.0000,2) Assoc w R - Interim,1.1476,...,0.6154,1.0000,0.0215,0.0723,0.9996,1.0000,77.3261,inf,0.9278,0.9786


In [25]:
who_variants.query("drug=='DLM' & confidence=='2) Assoc w R - Interim'")

,Unnamed: 0,drug,genome_index,confidence,gene,variant
5248,3804,DLM,3986989,2) Assoc w R - Interim,ddn,ddn_L49P


In [26]:
who_variants.query("drug=='DLM'").confidence.unique()

array(['3) Uncertain significance', '2) Assoc w R - Interim'],
      dtype=object)

In [11]:
mxf_analysis.query("Tier1_only==0 & WHO_phenos==0 & Syn==1")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,OR_LB,OR_UB,Tier1_only,WHO_phenos,poolLOF,Syn
370,Rv2752c_c.675C>T,0.1101,0.0470,0.1568,0.0000,0.0009,0.1172,NaN,NaN,1.1164,1.0481,1.1698,0,0,1,1
371,glpK_lof,0.1000,0.0384,0.1619,0.0008,0.0085,1.0000,NaN,NaN,1.1052,1.0392,1.1758,0,0,1,1
372,gyrA_c.2022C>G,0.0618,0.0000,0.0726,0.0063,0.0339,1.0000,NaN,NaN,1.0637,1.0000,1.0753,0,0,1,1
373,gyrA_p.Arg392Cys,0.0615,0.0000,0.0737,0.0037,0.0278,1.0000,NaN,NaN,1.0634,1.0000,1.0764,0,0,1,1
374,glpK_c.117G>C,0.0603,-0.0284,0.0948,0.0310,0.0954,1.0000,NaN,NaN,1.0621,0.9720,1.0995,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,glpK_p.Leu228Val,-0.0906,-0.1457,-0.0385,0.0018,0.0170,1.0000,NaN,NaN,0.9134,0.8644,0.9622,0,0,1,1
2520,Rv2477c_c.1545C>T,-0.1315,-0.2418,-0.0283,0.0088,0.0421,1.0000,NaN,NaN,0.8767,0.7852,0.9721,0,0,1,1
2521,Rv2477c_p.Gly41Glu,-0.1405,-0.2298,-0.0660,0.0005,0.0057,1.0000,NaN,NaN,0.8689,0.7947,0.9361,0,0,1,1
2522,glpK_c.1134C>T,-0.1410,-0.2736,-0.0166,0.0185,0.0749,1.0000,NaN,NaN,0.8685,0.7606,0.9836,0,0,1,1


In [4]:
mxf_analysis.Tier1_only.value_counts()

0    2271
1     406
Name: Tier1_only, dtype: int64

In [5]:
mxf_analysis.WHO_phenos.value_counts()

0    2307
1     370
Name: WHO_phenos, dtype: int64

In [6]:
mxf_analysis.poolLOF.value_counts()

1    2524
0     153
Name: poolLOF, dtype: int64

In [7]:
mxf_analysis.Syn.value_counts()

1    2176
0     501
Name: Syn, dtype: int64

# Catalog-Based Method Comparison

In [20]:
def get_logReg_summary(out_dir):
    
    model_summary = pd.read_csv(os.path.join(out_dir, "logReg_summary.csv"))
    model_analysis = pd.read_csv(os.path.join(out_dir, "model_analysis.csv"))
    
    summary_sens = model_summary.loc[0, "Sens"]
    summary_spec = model_summary.loc[0, "Spec"]
    
    #max_idx = [np.argmax(model_analysis["Sens"]), np.argmax(model_analysis["Spec"])]
    
    better_variant = model_analysis.query("Sens >= @summary_sens & Spec >= @summary_spec")
    
    if len(better_variant) == 0:
        print("No variants have comparable sensitivity AND specificity")
    else:
        print(better_variant)
    return model_summary, model_analysis
    
    #return model_summary, model_analysis.iloc[max_idx, :][["orig_variant", "coef", "Sens", "Spec", "accuracy", "balanced_accuracy"]]

In [26]:
#out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Levofloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn"

summary, analysis = get_logReg_summary(out_dir)
summary

No variants have comparable sensitivity AND specificity


,Sens,Spec,accuracy,balanced_accuracy
0,0.8336,0.9084,0.8951,0.8710


In [27]:
analysis.query("Sens > 0.9006")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,accuracy,balanced_accuracy


In [29]:
analysis.loc[analysis["orig_variant"].str.contains("PC")]

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,accuracy,balanced_accuracy
35,PC2,0.1660,0.0355,0.3034,0.0078,0.0301,1.0000,NaN,NaN,1.1806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,PC0,0.1346,0.0166,0.2630,0.0140,0.0492,1.0000,NaN,NaN,1.1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
analysis.query("Spec > 0.9575")

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,genome_index,confidence_WHO_2021,Odds_Ratio,...,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,accuracy,balanced_accuracy
0,pncA_c.-11A>G,0.4088,0.3700,0.4467,0.0000,0.0000,0.0000,2289252.0000,1) Assoc w R,1.5050,...,0.0274,0.0395,0.9993,0.9999,44.4364,284.1349,0.9608,0.9731,0.8278,0.5163
1,pncA_p.His51Asp,0.3308,0.3054,0.3591,0.0000,0.0000,0.0000,2289091.0000,1) Assoc w R,1.3921,...,0.0095,0.0177,0.9998,1.0000,63.1368,inf,0.9823,0.9905,0.8246,0.5067
2,pncA_p.Val131fs,0.2900,0.2684,0.3154,0.0000,0.0000,0.0000,NaN,NaN,1.3365,...,0.0090,0.0165,0.9998,1.0000,54.4333,inf,0.9836,0.9912,0.8245,0.5064
3,pncA_p.Gln141Pro,0.2869,0.2529,0.3273,0.0000,0.0000,0.0000,2288820.0000,1) Assoc w R,1.3323,...,0.0151,0.0243,0.9995,0.9999,35.3919,344.8754,0.9759,0.9851,0.8256,0.5097
4,pncA_p.Leu172Pro,0.2779,0.2524,0.3046,0.0000,0.0000,0.0000,2288727.0000,1) Assoc w R,1.3204,...,0.0062,0.0129,1.0000,1.0000,inf,inf,0.9871,0.9938,0.8239,0.5046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,rpsA_p.Ile70Leu,-0.1249,-0.1356,-0.1120,0.0000,0.0000,0.0000,NaN,NaN,0.8826,...,0.0000,0.0000,0.9966,0.9982,0.0000,0.0000,1.0018,1.0034,0.1798,0.5013
855,PPE35_p.Pro670Leu,-0.1287,-0.1817,-0.0860,0.0000,0.0000,0.0009,2168604.0000,5) Not assoc w R,0.8792,...,0.0003,0.0024,0.9800,0.9842,0.0156,0.1377,1.0145,1.0196,0.1922,0.5084
856,Rv1258c_p.Gly363Val,-0.1288,-0.1399,-0.1158,0.0000,0.0000,0.0000,1406253.0000,3) Uncertain significance,0.8791,...,0.0000,0.0000,0.9965,0.9981,0.0000,0.0000,1.0019,1.0036,0.1799,0.5013
857,Rv3236c_c.546C>T,-0.1577,-0.1859,-0.1337,0.0000,0.0000,0.0000,NaN,NaN,0.8541,...,0.0000,0.0000,0.9916,0.9942,0.0000,0.0000,1.0058,1.0085,0.1835,0.5035


In [19]:
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Moxifloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"

get_logReg_summary(out_dir)

No variants have comparable sensitivity AND specificity


,Sens,Spec,accuracy,balanced_accuracy
0,0.9230,0.8945,0.8983,0.9088


In [38]:
df_pza = pd.read_csv("/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn/phenos.csv")

# the missing ones might be M. cannettii, most similar to L6 based on the other lineage callers
lineages = pd.read_pickle("../data/combined_lineage_sample_IDs.pkl")
lineages["Lineage"] = lineages["Lineage"].fillna("6")
lineages["Lineage_1"] = lineages["Lineage_1"].fillna("6")

lineages = lineages[["Sample Name", "Sample ID", "Lineage_1"]]
lineages["Lineage"] = [str(val).split(".")[0] for val in lineages["Lineage_1"].values]
lineages.loc[lineages["Lineage"].str.contains("BOV"), "Lineage"] = "M. bovis"

assert len(lineages.loc[pd.isnull(lineages["Lineage"])]) == 0

########## KEEP ONLY ISOLATES WITH ALL 3 PIECES OF DATA ##########

# get only isolates with data for everyting: SNP matrix, in the model, and lineages
combined = lineages.merge(df_pza, left_on="Sample ID", right_on="sample_id")

In [42]:
combined.groupby("Lineage")["phenotype"].mean().sort_values(ascending=False)

Lineage
M. bovis   0.9153
2          0.3750
1          0.1503
6          0.1163
4          0.1112
5          0.1111
3          0.0554
7          0.0000
Name: phenotype, dtype: float64

In [45]:
combined.query("Lineage == 'M. bovis'").Lineage_1.unique()

array(['BOV_AFRI', 'BOV'], dtype=object)